In [1]:
!pip install tweepy
!pip install preprocessor
!pip install textblob

In [2]:
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# importing libraries
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import urlopen
import re
import time

import pandas as pd #Importing the PANDAS python library
import numpy as np #importing Numpy
%matplotlib inline 

#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #initiating VADER instance

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/randallgeszvain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/randallgeszvain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/randallgeszvain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
#Twitter credentials for the app
# Consume:
CONSUMER_KEY    = '9BKdl1VWnF0HAatcsLnlAMRgy'
CONSUMER_SECRET = 'X4AhMhjkdqLsuOTdpVmzOhOh87oM44KBLwa0sYBA0BQWCT6Y3L'

# Access:
ACCESS_TOKEN  = '1124830893934370816-lOoMmIY66jAfrln1kWUOgzRgYRFOsU'
ACCESS_SECRET = 'oSTrJkdwGRVjLtTTP8p1jQLt7yYliO95Uxez8XQqs9oX1'

In [4]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

In [5]:
#declare file paths as follows for three files
AgeofUltron_tweets = "AgeofUltron_data.csv"
Antman_tweets = "Antman_data.csv"
CivilWar_tweets = "CivilWar_data.csv"
DoctorStrange_tweets = "DoctorStrange_data.csv"
GuardiansoftheGalaxy_tweets = "GuardiansoftheGalaxy_data.csv"
SpidermanHomecoming_tweets = "SpidermanHomecoming_data.csv"
ThorRagnarok_tweets = "ThorRagnarok_data.csv"
BlackPanther_tweets = "BlackPanther_data.csv"
AvengersInfinityWar_tweets = "AvengersInfinityWar_data.csv"
Antmanthewasp_tweets = "AntMantheWasp_data.csv"
AvengersEndgame_tweets = "AvengersEndgame_data.csv"
RogueOne_tweets = "RogueOne_data.csv"
Solo_tweets = "Solo_data.csv"
TheForceAwakens_tweets = "TheForceAwakens_data.csv"
TheLastJedi_tweets = "TheLastJedi_data.csv"
DawnofJustice_tweets = "DawnofJustice_data.csv"
SuicideSquad_tweets = "SuicideSquad_data.csv"
WonderWoman_tweets = "WonderWoman_data.csv"
JusticeLeague_tweets = "JusticeLeague_data.csv"
Aquaman_tweets = "Aquaman_data.csv"
Shazam_tweets = "Shazam_data.csv"
SpidermanFarfromhome_tweets = "SpidermanFarfromhome_data.csv"

In [6]:
#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
        'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
        'user_mentions', 'place', 'place_coord_boundaries']
 
#set two date variables for date range
start_date = '2015-01-01'
end_date = '2019-06-12'

In [7]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

In [8]:
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

In [9]:
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [10]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [11]:
#mrhod clean_tweets()
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    #Add stop words
    stop_words.add("AvengersEndgame")
    stop_words.add("Avengers")
    stop_words.add("Endgame")
    stop_words.add("Marvel")
    stop_words.add("Studio")
    stop_words.add("Avengersâ")
    word_tokens = word_tokenize(tweet)
 
    #after tweepy preprocessing the colon left remain after removing mentions
    #or RT sign in the beginning of the tweet
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
 
 
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
 
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
 
    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)

In [12]:
#method write_tweets()
def write_tweets(keyword, file):
    # If the file exists, then read the existing data from the CSV file.
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    #page attribute in tweepy.cursor and iteration
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=start_date).pages(50):
        for status in page:
            new_entry = []
            status = status._json
 
            ## check whether the tweet is in english or skip to the next tweet
            if status['lang'] != 'en':
                continue
 
            #when run the code, below code replaces the retweet amount and
            #no of favorires that are changed since last download.
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue
 
 
           #tweepy preprocessing called for basic preprocessing
            clean_text = status['text']
 
            #call clean_tweet method for extra preprocessing
            filtered_tweet=clean_tweets(clean_text)
 
            #pass textBlob method for sentiment calculations
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment
 
            #seperate polarity and subjectivity in to two variables
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
 
            #new entry append
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]
 
            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])
 
            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)
 
            # hashtagas and mentiones are saved using comma separted
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)
 
            #get location of the tweet if possible
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)
 
            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)
 
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")

In [13]:
#declare keywords as a query for three categories
#AgeofUltron_keywords = '#AgeofUltron OR #AvengersAgeofUltron'
#Antman_keywords = '#Antman'
#CivilWar_keywords = '#CivilWar OR #CaptainAmericaCivilWar'
#DoctorStrange_keywords = '#DoctorStrange'
#GuardiansoftheGalaxy_keywords = '#GuardiansoftheGalaxy'
#SpidermanHomecoming_keywords = '#SpidermanHomecoming'
#ThorRagnarok_keywords = '#ThorRagnarok OR #Thor'
#BlackPanther_keywords = '#BlackPanther'
#AvengersInfinityWar_keywords = '#InfinityWar OR #AvengersInfinityWar'
#Antmanthewasp_keywords = '#Antmanthewasp OR #thewasp'
#AvengersEndgame_keywords = '#AvengersEndgame OR #Endgame'
#RogueOne_keywords = '#RogueOne OR #StarWarsRogueOne'
#Solo_keywords = '#StarWarsSolo'
#TheForceAwakens_keywords = '#TheForceAwakens OR #ForceAwakens'
#TheLastJedi_keywords = '#TheLastJedi OR #LastJedi'
#DawnofJustice_keywords = '#DawnofJustice OR #BatmanvSuperman OR #BatmanvsSuperman'
#SuicideSquad_keywords = '#SuicideSquad'
#WonderWoman_keywords = '#WonderWoman'
#JusticeLeague_keywords = '#JusticeLeague'
#Aquaman_keywords = '#Aquaman'
#Shazam_keywords = '#Shazam'
SpidermanFarfromhome_keywords = '#SpidermanFarfromhome'



#call main method passing keywords and file path
#write_tweets(AgeofUltron_keywords,  AgeofUltron_tweets)
#write_tweets(Antman_keywords, Antman_tweets)
#write_tweets(CivilWar_keywords, CivilWar_tweets)
#write_tweets(DoctorStrange_keywords, DoctorStrange_tweets)
#write_tweets(GuardiansoftheGalaxy_keywords, GuardiansoftheGalaxy_tweets)
#write_tweets(SpidermanHomecoming_keywords, SpidermanHomecoming_tweets)
#write_tweets(ThorRagnarok_keywords, ThorRagnarok_tweets)
#write_tweets(BlackPanther_keywords, BlackPanther_tweets)
#write_tweets(AvengersInfinityWar_keywords, AvengersInfinityWar_tweets)
#write_tweets(Antmanthewasp_keywords, Antmanthewasp_tweets)
#write_tweets(AvengersEndgame_keywords, AvengersEndgame_tweets)
#write_tweets(RogueOne_keywords, RogueOne_tweets)
#write_tweets(Solo_keywords, Solo_tweets)
#write_tweets(TheForceAwakens_keywords, TheForceAwakens_tweets)
#write_tweets(TheLastJedi_keywords, TheLastJedi_tweets)
#write_tweets(DawnofJustice_keywords, DawnofJustice_tweets)
#write_tweets(SuicideSquad_keywords, SuicideSquad_tweets)
#write_tweets(WonderWoman_keywords, WonderWoman_tweets)
#write_tweets(JusticeLeague_keywords, JusticeLeague_tweets)
#write_tweets(Aquaman_keywords, Aquaman_tweets)
#write_tweets(Shazam_keywords, Shazam_tweets)
write_tweets(SpidermanFarfromhome_keywords, SpidermanFarfromhome_tweets)
